In [1]:
utils_path = '/Users/aaronwu/Documents/misc/tmp/selenium/'
import sys
sys.path.append(utils_path)

from selenium_helper import *
from moonboard_helper import *

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import shutil
import pickle
import os
import pdb
import time

In [3]:
def click_next_page(driver, current_page):
    next_page=current_page+1
    pager = None
    clicked = False
    spans = driver.find_elements_by_tag_name('span')
    for s in spans:
        if s.get_attribute('class')=='k-link k-pager-nav':
            pager=s
    if pager==None:
        return clicked
    pager.click()
    items = driver.find_elements_by_tag_name('a')
    for i in items:
        if i.get_attribute('data-page')==str(next_page):
            i.click()
            return True
    return False

def get_repeats_data(driver):
    num_tries = 20
    repeats_data = []
    attr_dict={'class':'repeats'}
    current_page=1
    while True:
        repeats_add=None
        for x in range(num_tries):
            try:
                repeats = get_elem_set(driver, 'div', attr_dict)
                for repeat in repeats:
                    h3_item = repeat.find_elements_by_tag_name('h3')[0]
                    user_name = h3_item.text
                    uid = h3_item.find_elements_by_tag_name('a')[0].get_attribute('href').split('/Account/Profile/')[1]
                    repeats_add = [r.text for r in repeat.find_elements_by_tag_name('p')] + [user_name, uid]
                    repeats_data.append(repeats_add)
                break
            except:
                time.sleep(.5)
                if x==num_tries-1:
                    pdb.set_trace()
                continue
            if repeats_add==None:
                break
            if len(remove_duplicates(repeats_add))<6:
                break
        if repeats_add==None:
            return None
        for x in range(num_tries):
            try:
                clicked = click_next_page(driver, current_page)
                break
            except:
                time.sleep(.5)
                continue
        if not clicked:
            break
        current_page+=1
    repeats_data = remove_duplicates(repeats_data)
    return repeats_data

def remove_duplicates(repeats_data):
    repeats = []
    for r in repeats_data:
        if not r in repeats:
            repeats.append(r)
    return repeats

def get_target_script(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    scripts = soup.find_all("script")
    for s in scripts:
        if 'JSON' in s.text:
            return s.text
    return ''
    
def parse_target_json(text, start="var problem = JSON.parse('", end="');"):
    json_dict = json.loads(text.split(start)[1].split(end)[0])
    return json_dict

def parse_moves(json_dict):
    return json_dict['Moves']

In [4]:
parent_path = '/Users/aaronwu/Documents/misc/tmp/selenium/'
executable_path = parent_path + 'chromedriver'
scraped_path = parent_path + 'moonboard_2016_output.pickle'
edited_path = parent_path + 'moonboard_2016_output_edited.pickle'
failed_uids_path = parent_path + 'moonboard_2016_failed_uids.pickle'

url = 'https://www.moonboard.com/'
username = 'aaronlwu257@hotmail.com'
password = 'temppass1992'

In [5]:
if not os.path.exists(edited_path):
    shutil.copyfile(scraped_path, edited_path)
problems_dict = pickle.load(open(edited_path,'rb'))

In [6]:
driver = load_driver(executable_path=executable_path)

In [7]:
loginMoonBoard(driver, url=url, username=username, password=password)

In [8]:
num_tries=10
uids = sorted(problems_dict.keys())
for i,uid in enumerate(uids):
    print(str(i)+'/'+str(len(uids)))
    problem = problems_dict[uid]
    if ('moves' in problem) and ('repeats' in problem):
        continue
    if check_fail_url(driver, problem['url']):
        print(str(uid) + ' failed')
        continue
    else:
        change=False
        driver.get(problem['url'])
        for x in range(num_tries):
            try:
                if not 'moves' in problem:
                    target_text = get_target_script(driver)
                    json_dict = parse_target_json(target_text)
                    problem['moves'] = parse_moves(json_dict)
                    change=True
                if not 'repeats' in problem:
                    problem['repeats'] = get_repeats_data(driver)
                    change=True
            except:
                print('Failed to find moves')
                time.sleep(.5)
                continue
        if change:
            problems_dict[uid] = problem
            pickle.dump(problems_dict, open(edited_path,'wb'))

0/13607
1/13607
2/13607
3/13607
4/13607
5/13607
6/13607
7/13607
8/13607
9/13607
10/13607
11/13607
12/13607
13/13607
14/13607
15/13607
16/13607
17/13607
18/13607
19/13607
20/13607
21/13607
22/13607
23/13607
24/13607
25/13607
26/13607
27/13607
28/13607
29/13607
30/13607
31/13607
32/13607
33/13607
34/13607
35/13607
36/13607
37/13607
38/13607
39/13607
40/13607
41/13607
42/13607
43/13607
44/13607
45/13607
46/13607
47/13607
48/13607
49/13607
50/13607
51/13607
52/13607
53/13607
54/13607
55/13607
56/13607
57/13607
58/13607
59/13607
60/13607
61/13607
62/13607
63/13607
64/13607
65/13607
66/13607
67/13607
68/13607
69/13607
70/13607
71/13607
72/13607
73/13607
74/13607
75/13607
76/13607
77/13607
78/13607
79/13607
80/13607
81/13607
82/13607
83/13607
84/13607
85/13607
86/13607
87/13607
88/13607
89/13607
90/13607
91/13607
92/13607
93/13607
94/13607
95/13607
96/13607
97/13607
98/13607
99/13607
100/13607
101/13607
102/13607
103/13607
104/13607
105/13607
106/13607
107/13607
108/13607
109/13607
110/13607


Failed to find class field-validation-error
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
1253/13607
1254/13607
1255/13607
1256/13607
1257/13607
1258/13607
1259/13607
1260/13607
1261/13607
1262/13607
1263/13607
1264/13607
1265/13607
1266/13607
1267/13607
1268/13607
1269/13607
1270/13607
1271/13607
1272/13607
1273/13607
1274/13607
1275/13607
1276/13607
1277/13607
1278/13607
1279/13607
1280/13607
1281/13607
1282/13607
1283/13607
1284/13607
1285/13607
1286/13607
1287/13607
1288/13607
1289/13607
1290/13607
1291/13607
1292/13607
1293/13607
1294/13607
1295/13607
1296/13607
1297/13607
1298/13607
1299/13607
1300/13607
1301/13607
1302/13607
1303/13607
1304/13607
1305/13607
1306/13607
1307/13607
1308/13607
1309/13607
1310/13607
1311/13607
1312/13607
1313/13607
1314/13607
1315/13607
1316/13607
1317/13607
1318/13607
1319/13607
1320/1360

Failed to find class field-validation-error
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
2129/13607
2130/13607
2131/13607
2132/13607
2133/13607
2134/13607
2135/13607
2136/13607
2137/13607
2138/13607
2139/13607
2140/13607
2141/13607
2142/13607
2143/13607
2144/13607
2145/13607
2146/13607
2147/13607
2148/13607
2149/13607
2150/13607
2151/13607
2152/13607
2153/13607
2154/13607
2155/13607
2156/13607
2157/13607
2158/13607
2159/13607
2160/13607
2161/13607
2162/13607
2163/13607
2164/13607
2165/13607
2166/13607
2167/13607
2168/13607
2169/13607
2170/13607
2171/13607
2172/13607
2173/13607
2174/13607
2175/13607
2176/13607
2177/13607
2178/13607
2179/13607
2180/13607
2181/13607
2182/13607
2183/13607
2184/13607
2185/13607
2186/13607
2187/13607
2188/13607
2189/13607
2190/13607
2191/13607
2192/13607
2193/13607
2194/13607
2195/13607
2196/1360

Failed to find class field-validation-error
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
2845/13607
2846/13607
2847/13607
2848/13607
2849/13607
2850/13607
2851/13607
2852/13607
2853/13607
2854/13607
2855/13607
2856/13607
2857/13607
2858/13607
2859/13607
2860/13607
2861/13607
2862/13607
2863/13607
2864/13607
2865/13607
2866/13607
2867/13607
2868/13607
2869/13607
2870/13607
2871/13607
2872/13607
2873/13607
2874/13607
2875/13607
2876/13607
2877/13607
2878/13607
2879/13607
2880/13607
2881/13607
2882/13607
2883/13607
2884/13607
2885/13607
2886/13607
2887/13607
2888/13607
2889/13607
2890/13607
2891/13607
2892/13607
2893/13607
2894/13607
2895/13607
2896/13607
2897/13607
2898/13607
2899/13607
2900/13607
2901/13607
2902/13607
2903/13607
2904/13607
2905/13607
2906/13607
2907/13607
2908/13607
2909/13607
2910/13607
2911/13607
2912/1360

Failed to find class field-validation-error
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
Failed to find moves
4658/13607
4659/13607
4660/13607
4661/13607
4662/13607
4663/13607
4664/13607
4665/13607
4666/13607
4667/13607
4668/13607
4669/13607
4670/13607
4671/13607
4672/13607
4673/13607
4674/13607
4675/13607
4676/13607
4677/13607
4678/13607
4679/13607
4680/13607
4681/13607
4682/13607
4683/13607
4684/13607
4685/13607
4686/13607
4687/13607
4688/13607
4689/13607
4690/13607
4691/13607
4692/13607
4693/13607
4694/13607
4695/13607
4696/13607
4697/13607
4698/13607
4699/13607
4700/13607
4701/13607
4702/13607
4703/13607
4704/13607
4705/13607
4706/13607
4707/13607
4708/13607
4709/13607
4710/13607
4711/13607
4712/13607
4713/13607
4714/13607
4715/13607
4716/13607
4717/13607
4718/13607
4719/13607
4720/13607
4721/13607
4722/13607
4723/13607
4724/13607
4725/1360

...

360973 failed
12736/13607
12737/13607
12738/13607
12739/13607
12740/13607
12741/13607
12742/13607
12743/13607
12744/13607
12745/13607
12746/13607
12747/13607
12748/13607
12749/13607
12750/13607
12751/13607
12752/13607
12753/13607
12754/13607
12755/13607
12756/13607
12757/13607
12758/13607
12759/13607
12760/13607
12761/13607
12762/13607
12763/13607
12764/13607
12765/13607
12766/13607
12767/13607
12768/13607
12769/13607
12770/13607
12771/13607
12772/13607
12773/13607
12774/13607
12775/13607
12776/13607
12777/13607
12778/13607
12779/13607
12780/13607
12781/13607
12782/13607
12783/13607
12784/13607
12785/13607
12786/13607
12787/13607
12788/13607
12789/13607
12790/13607
12791/13607
12792/13607
12793/13607
12794/13607
12795/13607
12796/13607
12797/13607
12798/13607
12799/13607
12800/13607
12801/13607
12802/13607
12803/13607
12804/13607
12805/13607
12806/13607
12807/13607
12808/13607
12809/13607
12810/13607
12811/13607
12812/13607
12813/13607
12814/13607
12815/13607
12816/13607
12817/13607
12

Failed to find class field-validation-error
13055/13607
Failed to find class field-validation-error
13056/13607
Failed to find class field-validation-error
13057/13607
Failed to find class field-validation-error
13058/13607
Failed to find class field-validation-error
13059/13607
Failed to find class field-validation-error
13060/13607
Failed to find class field-validation-error
13061/13607
Failed to find class field-validation-error
13062/13607
Failed to find class field-validation-error
13063/13607
Failed to find class field-validation-error
13064/13607
Failed to find class field-validation-error
13065/13607
Failed to find class field-validation-error
13066/13607
Failed to find class field-validation-error
13067/13607
Failed to find class field-validation-error
13068/13607
Failed to find class field-validation-error
13069/13607
Failed to find class field-validation-error
13070/13607
Failed to find class field-validation-error
13071/13607
Failed to find class field-validation-error
1307

13201/13607
Failed to find class field-validation-error
13202/13607
Failed to find class field-validation-error
13203/13607
Failed to find class field-validation-error
13204/13607
Failed to find class field-validation-error
13205/13607
Failed to find class field-validation-error
13206/13607
Failed to find class field-validation-error
13207/13607
Failed to find class field-validation-error
13208/13607
Failed to find class field-validation-error
13209/13607
Failed to find class field-validation-error
13210/13607
Failed to find class field-validation-error
13211/13607
Failed to find class field-validation-error
13212/13607
Failed to find class field-validation-error
13213/13607
Failed to find class field-validation-error
13214/13607
Failed to find class field-validation-error
13215/13607
Failed to find class field-validation-error
13216/13607
Failed to find class field-validation-error
13217/13607
Failed to find class field-validation-error
13218/13607
Failed to find class field-validatio

13348/13607
Failed to find class field-validation-error
13349/13607
Failed to find class field-validation-error
13350/13607
Failed to find class field-validation-error
13351/13607
Failed to find class field-validation-error
13352/13607
Failed to find class field-validation-error
13353/13607
Failed to find class field-validation-error
13354/13607
Failed to find class field-validation-error
13355/13607
Failed to find class field-validation-error
13356/13607
Failed to find class field-validation-error
13357/13607
Failed to find class field-validation-error
13358/13607
Failed to find class field-validation-error
13359/13607
Failed to find class field-validation-error
13360/13607
Failed to find class field-validation-error
13361/13607
Failed to find class field-validation-error
13362/13607
Failed to find class field-validation-error
13363/13607
Failed to find class field-validation-error
13364/13607
Failed to find class field-validation-error
13365/13607
Failed to find class field-validatio

13495/13607
Failed to find class field-validation-error
13496/13607
Failed to find class field-validation-error
13497/13607
Failed to find class field-validation-error
13498/13607
Failed to find class field-validation-error
13499/13607
Failed to find class field-validation-error
13500/13607
Failed to find class field-validation-error
13501/13607
Failed to find class field-validation-error
13502/13607
Failed to find class field-validation-error
13503/13607
Failed to find class field-validation-error
13504/13607
Failed to find class field-validation-error
13505/13607
Failed to find class field-validation-error
13506/13607
Failed to find class field-validation-error
13507/13607
Failed to find class field-validation-error
13508/13607
Failed to find class field-validation-error
13509/13607
Failed to find class field-validation-error
13510/13607
Failed to find class field-validation-error
13511/13607
Failed to find class field-validation-error
13512/13607
Failed to find class field-validatio